In [46]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div([
   
    html.A(
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode('utf-8')),
            style={
                'height': '150px',
                'display': 'block',
                'margin': '0 auto'
            }
        ),
        href="https://www.snhu.edu",
        target="_blank"
    ),

    html.Div([
        html.Div("Jason Carignan", style={"flex": "1", "textAlign": "left", "fontWeight": "bold"}),
        html.Div("CS 340 – Project Two Dashboard", style={"flex": "1", "textAlign": "center", "fontWeight": "bold"}),
        html.Div("10/16/2025", style={"flex": "1", "textAlign": "right", "fontWeight": "bold"})
    ],
    style={
        "display": "flex",
        "justifyContent": "space-between",
        "alignItems": "center",
        "marginTop": "10px",
        "marginBottom": "10px"
    }),

    html.Hr(),
]),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        [
            dcc.RadioItems(
                id="filter-type",
                options=[
                    {"label": "Water Rescue", "value": "water"},
                    {"label": "Mountain Rescue", "value": "mountain"},
                    {"label": "Disaster Rescue", "value": "disaster"},
                    {"label": "Reset", "value": "reset"},
                ],
                value="reset",
                inline=True
            )
        ]
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
                            filter_action="native",
                            sort_action="native",
                            page_action="native",
                            page_current=0,
                            page_size=10,
                            column_selectable="single",
                            row_selectable="single",
                            selected_rows=[0],
                            style_table={'overflowX': 'auto'},
                            style_cell={
                                'textAlign': 'left',
                                'padding': '5px',
                                'whiteSpace': 'normal',
                                'height': 'auto',
        },
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            ), 
           html.H4("Jason Carignan") #Added my name to the footer for screenshots.      
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
## FIX ME Add code to filter interactive data table with MongoDB queries

    if filter_type == "reset":
        query = {}  # Return all without a filter
    elif filter_type == "water": # Water rescue as found in table
        query = {
            "$and": [
                {"animal_type": "Dog"},
                {"sex_upon_outcome": "Intact Female"},
                {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}},
                {"$or": [
                    {"breed": {"$regex": "Labrador Retriever", "$options": "i"}},
                    {"breed": {"$regex": "Chesapeake Bay Retriever", "$options": "i"}},
                    {"breed": {"$regex": "Newfoundland", "$options": "i"}}
                ]}
            ]
        }
    elif filter_type == "mountain":
        query = {
            "$and": [
                {"animal_type": "Dog"},
                {"sex_upon_outcome": "Intact Male"},
                {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}},
                {"$or": [
                    {"breed": {"$regex": "German Shepherd", "$options": "i"}},
                    {"breed": {"$regex": "Alaskan Malamute", "$options": "i"}},
                    {"breed": {"$regex": "Old English Sheepdog", "$options": "i"}},
                    {"breed": {"$regex": "Siberian Husky", "$options": "i"}},
                    {"breed": {"$regex": "Rottweiler", "$options": "i"}}
                ]}
            ]
        }
    elif filter_type == "disaster":
        query = {
            "$and": [
                {"animal_type": "Dog"},
                {"sex_upon_outcome": "Intact Male"},
                {"age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}},
                {"$or": [
                    {"breed": {"$regex": "Doberman Pinscher", "$options": "i"}},
                    {"breed": {"$regex": "German Shepherd", "$options": "i"}},
                    {"breed": {"$regex": "Golden Retriever", "$options": "i"}},
                    {"breed": {"$regex": "Bloodhound", "$options": "i"}},
                    {"breed": {"$regex": "Rottweiler", "$options": "i"}}
                ]}
            ]
        }
    else:
        query = {"animal_type": "Dog"}

    # Read the query from MongoDB
    df_local = pd.DataFrame.from_records(db.read(query))
    if not df_local.empty:
        df_local.drop(columns=['_id'], inplace=True, errors='ignore')

    # Update data based on tselection
    return df_local.to_dict('records')

#
#        
#        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
#        data=df.to_dict('records')
#       
#       
#        return (data,columns)


# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    if not viewData:
        return [html.Div("No Data")]
    
    dff = pd.DataFrame(viewData)
    
    if "animal_type" in dff.columns:
        dff = dff[dff["animal_type"].str.lower() == "dog"]
    
    if dff.empty or "breed" not in dff.columns:
        return [html.Div("No Data")]
    
    # Count breeds
    counts = dff["breed"].value_counts()

    # If more than 10 breeds, keep top 10 and bundle the rest as "Other"
    if counts.size > 10:
        top10 = counts.iloc[:10]
        other_sum = counts.iloc[10:].sum()
        breed_counts = top10.reset_index()
        breed_counts.columns = ["breed", "count"]
        if other_sum > 0:
            breed_counts = pd.concat(
                [breed_counts, pd.DataFrame([{"breed": "Other", "count": other_sum}])],
                ignore_index=True
            )
        title = "Dog Breeds"
    else:
        breed_counts = counts.reset_index()
        breed_counts.columns = ["breed", "count"]
        title = "Dog Breeds Distribution"

    fig = px.pie(breed_counts, names="breed", values="count", title=title)
    fig.update_traces(textposition="inside", textinfo="percent+label")

    return [dcc.Graph(figure=fig)]
        
    #return [
    #    dcc.Graph(            
    #        figure = px.pie(df, names='breed', title='Preferred Animals')
    #    )    
    #]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    # Updated to prevent callback error on load. This returns a list even if no column has been selected yet.
    if not selected_columns:
        return [] 
    
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if not viewData:
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'},
                center=[30.75,-97.48], zoom=10,
                children=[dl.TileLayer(id="base-layer-id")])
    ]
        
    dff = pd.DataFrame.from_dict(viewData)
    
    # Because we only allow single row selection, the list can be converted to a row index
    if index is None:
        row = 0
    else:
        row = index[0]
    
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
           center=[30.75,-97.48], zoom=10, children=[
           dl.TileLayer(id="base-layer-id"),
           # Marker with tool tip and popup
           # Column 13 and 14 define the grid-coordinates for 
           # the map
           # Column 4 defines the breed for the animal
           # Column 9 defines the name of the animal
           dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
              children=[
              dl.Tooltip(dff.iloc[row,4]),
              dl.Popup([
                 html.H1("Animal Name"),
                html.P(dff.iloc[row,9])
             ])
          ])
       ])
    ]
# Run ap

# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server() 

Dash app running on https://ivanapropos-originpage-3000.codio.io/proxy/8050/
